In [2]:
import requests
from bs4 import BeautifulSoup as bs
import re
from functools import lru_cache

In [30]:
def get_leaders():
    global countries
    root_url = "https://country-leaders.herokuapp.com"
    cookie_url = root_url + "/cookie"
    country_url = root_url + "/countries"
    leaders_url = root_url + "/leaders"

    req_cookies = requests.get(cookie_url)
    cookies=req_cookies.cookies
   
    req_countries = requests.get(country_url, cookies = cookies)
    countries = req_countries.text
#     print(countries)
    
    countries = countries.strip('[, ]')
    countries = countries.split(",")    
    
    session = requests.Session()
    leaders_per_country = {}
    for country in countries:
        country = country.replace('\"', "")
        param = {'country': country}
        
        req_leaders = requests.get(leaders_url, cookies =cookies, params = param)
        
        if req_leaders.status_code == 403:
            cookies=req_cookies.cookies
            req_leaders = requests.get(leaders_url, cookies =cookies, params = param)
            
        content = req_leaders.text    
        content = content.strip('[, ]')
        list_leaders_currentcountry = content.split('}')
#         print(".......... \nleaders info started here \n")
        clean_leader_info_percountry = []
        for leader_info in list_leaders_currentcountry:
            leader_info_clean = leader_info.strip('{, }')
            list_leader_info = leader_info_clean.split(',')
            leader_fname = ""
            leader_lname = ""
            wiki_url = ""
            
            leader_info_dict={}
            
            for info in list_leader_info:                
                if 'wikipedia' in info:
                    wiki_split = info.split(':')
                    wiki_url = (wiki_split[1] + ":" + wiki_split[2]).replace("\"", "")
                elif "first_name" in info:
                    leader_fname = info.split(':')[1].replace("\"", "")
                    
                elif "last_name" in info:
                    leader_lname = info.split(':')[1].replace("\"", "")
                    
                
                if leader_fname == "" or leader_lname == "" or wiki_url == "":
                    continue
                else:
                    break #has to break out from this loop because no neet to travel to all information
            print(leader_fname + " " + leader_lname + ",  " + wiki_url)
            #break
            try: 
                first_paragraph = get_first_paragraph_withCashWrapper(wiki_url, session)  
            except:
                first_paragraph = "Information could not be extracted. Either link not found or link has error"
                  
            print(first_paragraph)
            leader_info_dict['first_name'] = leader_fname
            leader_info_dict['last_name'] = leader_lname
            leader_info_dict['wikipedia_url'] = wiki_url
            leader_info_dict['first_paragraph'] = first_paragraph
            
            clean_leader_info_percountry.append(leader_info_dict)
        leaders_per_country[country] = clean_leader_info_percountry
    return leaders_per_country        
   



In [169]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

In [18]:
#@hashable_cache
@lru_cache(maxsize = 128)
def get_first_paragraph_withCashWrapper(wikipedia_url, session_param):
#     print(wikipedia_url)
    req= session_param.get(wikipedia_url) # requests changed by session_param
    content = req.text
    soup = bs(content, 'html')
    
    #remove all the text link
    for a in soup.findAll('a', href=True):
        a.extract()
        
    paragraphs = soup.find_all('p')
    first_paragraph_index = 0
    i = 0
    for paragraph in soup.find_all("p"):   
        if paragraph.find('b') != None:
            first_paragraph_index = i            
            break
        i+=1
    
    first_paragraph = paragraphs[first_paragraph_index].text
    
    first_paragraph = re.sub(r'[();{}[\]]+', "", first_paragraph)
    sanitized_paragraph = ' '.join(first_paragraph.strip().split())
    sanitized_paragraph = re.sub(r'II', 'II,',sanitized_paragraph)
   
    return sanitized_paragraph

In [9]:

s= requests.Session()
p = get_first_paragraph_withCashWrapper("https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD", s)
print(p)


https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD
Бори́с Никола́евич Е́льцин 1931-02-01, , , , — , , — и , государственный и политический деятель, первый всенародно избранный в ноябре 1991 — июне 1992 года одновременно возглавлял . С марта по май 1992 года исполнял обязанности .


In [218]:
# %%time
# leaders_per_country = get_leaders()

## Saving the work



In [33]:
import json

In [34]:
# 3 lines
for country in countries:
    country = country.replace('\"', "")
    file_name = "C:/BeCode/LocalRepos/GNT-Arai-4/" + country + "_" + "leaders.json"
    
    json_file = open(file_name, 'w')
    json_file.write(json.dumps(leaders_per_country.get(country)))
    json_file.close()
#     break




Make a function `save()` to call this code easily.

In [ ]:

# file_json = open(r"C:\BeCode\LocalRepos\GNT-Arai-4\us_leaders.json", 'r')
# data = json.load(file_json)
# print(data)
    
# file_json.close()

In [ ]:
# Call the function
# get_leaders()